In [90]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [91]:
pip install pyspark

In [92]:
pip install findspark

In [93]:
%config Completer.use_jedi = False

# Initialise the entry point for the spark

In [94]:
#from pyspark.sql import SparkSession

#spark = SparkSession.getOrCreate()
#data = spark.read.csv('/kaggle/input/hollywood-theatrical-market-synopsis-1995-to-2021/TopGrossingRatings.csv')
#data.show(10)

from pyspark.sql import SparkSession
  
spark = SparkSession.builder.appName("DataFrame").getOrCreate()





In [95]:
data = spark.read.option("header",True).csv('/kaggle/input/hollywood-theatrical-market-synopsis-1995-to-2021/TopGrossingRatings.csv')

# Initialising the entry point for the spark

In [96]:
data.select("MPAA RATINGS").show()
data.select("*").show()

* spark.catalog.dropTempView("dat")
    > remove temp view from memory if it already present
* data.createTempView("dat")'
>       register a temporary view in the memory
        createTempView(name) method of pyspark.sql.dataframe.DataFrame instance
        Creates a local temporary view with this :class:`DataFrame`.
>        
>         The lifetime of this temporary table is tied to the :class:`SparkSession`
        that was used to create this :class:`DataFrame`.
        throws :class:`TempTableAlreadyExistsException`, if the view name already exists in the
        catalog <

        

In [97]:
spark.catalog.dropTempView("dat")
data.createTempView("dat")
distinct = spark.sql("select count(distinct `MPAA RATINGS`) as count from dat")
distinct.show()

* data.printSchema() will print the schema of the dataframe

> Help on method printSchema in module pyspark.sql.dataframe:
> 
> printSchema() method of pyspark.sql.dataframe.DataFrame instance
>     Prints out the schema in the tree format.
>     
>     .. versionadded:: 1.3.0
>     
>     Examples
>     --------
>     >>> df.printSchema()
>     root
>      |-- age: integer (nullable = true)
>      |-- name: string (nullable = true)
>     <BLANKLINE>


In [98]:
data.printSchema()

* data.withColumn

>Help on method withColumn in module pyspark.sql.dataframe:
>
>**withColumn(colName, col)** method of pyspark.sql.dataframe.DataFrame instance
>    Returns a new :class:`DataFrame` by adding a column or replacing the
>    existing column that has the same name.
>    
>    The column expression must be an expression over this :class:`DataFrame`; attempting to add
>    a column from some other :class:`DataFrame` will raise an error.
>    
>    .. versionadded:: 1.3.0
>    
>    **Parameters**
>    
>    colName : str
>        string, name of the new column.
>    col : :class:`Column`
>        a :class:`Column` expression for the new column.
>    
>    **Notes**
>    
>    This method introduces a projection internally. Therefore, calling it multiple
>    times, for instance, via loops in order to add multiple columns can generate big
>    plans which can cause performance issues and even `StackOverflowException`.
>    To avoid this, use :func:`select` with the multiple columns at once.
>    
>    **Examples**
>   
>     df.withColumn('age2', df.age + 2).collect()
>
>    [Row(age=2, name='Alice', age2=4), Row(age=5, name='Bob', age2=7)]
>
>> This function can be used for data cleaning and data tyoecasting

In [99]:
help(data.withColumn)

* from pyspark.sql import functions as func
> functions contain predefined function which xan be used in columns in dataframe

In [100]:
from pyspark.sql import functions as func

* func.regexp_replace
>Help on function regexp_replace in module pyspark.sql.functions:
>
>regexp_replace(str, pattern, replacement)
>    Replace all substrings of the specified string value that match regexp with rep.
>    
>    .. versionadded:: 1.5.0
>    
>    Examples
>    --------
>     df = spark.createDataFrame([('100-200',)], ['str'])
>     df.select(regexp_replace('str', r'(\d+)', '--').alias('d')).collect()
>    [Row(d='-----')]
>

In [101]:
help(func.regexp_replace)

In [102]:
data.printSchema()
data1 = data.withColumn("MOVIES",func.regexp_replace(data.MOVIES,",",""))\
    .withColumn("TOTAL GROSS",func.regexp_replace(data["TOTAL GROSS"],"[\$,]","")) \
    .withColumn("AVERAGE GROSS",func.regexp_replace(data["AVERAGE GROSS"],"[\$,]",""))  \
    .withColumn("MARKET SHARE",func.regexp_replace(data["MARKET SHARE"],"[%]",""))    \
    

In [103]:
data1.show()

In [104]:

data.withColumn("MOVIES",func.regexp_replace(data.MOVIES,",",""))\
    .withColumn("TOTAL GROSS",func.regexp_replace(data["TOTAL GROSS"],"[\$,]","")) \
    .withColumn("AVERAGE GROSS",func.regexp_replace(data["AVERAGE GROSS"],"[\$,]",""))  \
    .withColumn("MARKET SHARE",func.regexp_replace(data["MARKET SHARE"],"[%]",""))   

In [105]:
help(func.col)

In [106]:
#using withColumn for type casting
data1=data1.withColumn("MOVIES",func.col("MOVIES").cast('int'))\
    .withColumn("TOTAL GROSS",func.col("TOTAL GROSS").cast('double')) \
    .withColumn("AVERAGE GROSS",func.col("AVERAGE GROSS").cast('double'))  \
    .withColumn("MARKET SHARE",func.col("MARKET SHARE").cast('double'))   

In [107]:
data1.printSchema()

In [108]:
data1.show()

In [109]:
data1.select("*").count()

In [110]:
data1.explain(mode="formatted")

In [111]:
data_sql = spark.sql("select RANK, \
          `MPAA RATINGS`, \
           cast(REGEXP_REPLACE(MOVIES,',','') as int) MOVIES, \
           cast(REGEXP_REPLACE(`TOTAL GROSS`,'[\$,]','') as double) `TOTAL GROSS`, \
           cast(REGEXP_REPLACE(`AVERAGE GROSS`,'[\$,]','') as double) `AVERAGE GROSS`, \
           cast(REGEXP_REPLACE(`MARKET SHARE`,'%','') as double) `MARKET SHARE` \
          from dat")
data_sql.explain(mode="formatted")

In [112]:
%%timeit
data_sql

In [113]:
%%timeit
data1